<a href="https://colab.research.google.com/github/esmmif/RPAD_Final/blob/main/RPAD_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Desafio 1: O Corretor de Imóveis (Regressão com XGBoost)

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

# 1. Carregar os dados
data = fetch_california_housing()
X, y = data.data, data.target

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Instanciar XGBRegressor
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

# Treinamento
model.fit(X_train, y_train)

# Predição
preds = model.predict(X_test)

# 3. Avaliação (RMSE e MAE)
# O target do dataset original é em unidades de $100.000, Precisamos ajustar para ter o erro real em dólares.
mae = mean_absolute_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f"--- Resultados da Regressão ---")
print(f"MAE (Unidades escaladas): {mae:.4f}")
print(f"RMSE (Unidades escaladas): {rmse:.4f}")
print(f"-"*30)
print(f"Erro Médio Absoluto (Real): ${mae * 100000:,.2f}")
print(f"Raiz do Erro Quadrático Médio (Real): ${rmse * 100000:,.2f}")

--- Resultados da Regressão ---
MAE (Unidades escaladas): 0.3155
RMSE (Unidades escaladas): 0.4767
------------------------------
Erro Médio Absoluto (Real): $31,545.10
Raiz do Erro Quadrático Médio (Real): $47,672.03


## Desafio 2: O Sommelier de Vinhos (Classificação Multiclasse)

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from xgboost import XGBClassifier

# 1. Carregar os dados
wine = load_wine()
X, y = wine.data, wine.target

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Configurar XGBClassifier para Multiclasse
# num_class=3 define o número de saídas da softmax.
model = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    n_estimators=100,
    random_state=42
)

# Treinamento
model.fit(X_train, y_train)

# Predição
y_pred = model.predict(X_test)

# 3. Matriz de Confusão e Acurácia
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print(f"--- Classificação de Vinhos ---")
print(f"Acurácia Global: {acc:.2%}")
print("\nMatriz de Confusão:")
print(cm)
print("\nAs linhas representam as classes reais e as colunas as preditas.")

--- Classificação de Vinhos ---
Acurácia Global: 96.30%

Matriz de Confusão:
[[19  0  0]
 [ 1 20  0]
 [ 0  1 13]]

As linhas representam as classes reais e as colunas as preditas.


## Desafio 3: O Detector de Fraudes (Dados Desbalanceados)

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, confusion_matrix
from xgboost import XGBClassifier

# 1. Gerar dados sintéticos desbalanceados (1% classe positiva)
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99, 0.01], random_state=42)

# Divisão com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# --- Modelo 1: Sem ajustes ---
model_default = XGBClassifier(random_state=42)
model_default.fit(X_train, y_train)
pred_default = model_default.predict(X_test)
recall_default = recall_score(y_test, pred_default)

# --- Modelo 2: Com scale_pos_weight ---
# Cálculo do peso: Negativos (Majoritária) / Positivos (Minoritária) ~ 99
weight = 99
model_weighted = XGBClassifier(scale_pos_weight=weight, random_state=42)
model_weighted.fit(X_train, y_train)
pred_weighted = model_weighted.predict(X_test)
recall_weighted = recall_score(y_test, pred_weighted)

print("--- Comparativo de Detecção de Fraude (Classe 1) ---")
print(f"Recall (Modelo Padrão): {recall_default:.2%} (Detectou {sum(pred_default)} fraudes)")
print(f"Recall (Com Peso):      {recall_weighted:.2%} (Detectou {sum(pred_weighted)} fraudes)")
print("\nNota: O aumento do Recall geralmente custa uma queda na Precisão (mais falsos positivos).")

--- Comparativo de Detecção de Fraude (Classe 1) ---
Recall (Modelo Padrão): 0.00% (Detectou 0 fraudes)
Recall (Com Peso):      25.00% (Detectou 2 fraudes)

Nota: O aumento do Recall geralmente custa uma queda na Precisão (mais falsos positivos).


## Desafio 4: Duelo de Modelos (Árvore vs. XGBoost)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Dataset
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuração
depth_limit = 3

# 1. Instanciar Modelos (Removido use_label_encoder)
dt_model = DecisionTreeClassifier(max_depth=depth_limit, random_state=42)
xgb_model = XGBClassifier(
    max_depth=depth_limit,
    n_estimators=100,
    random_state=42
)

# 2. Treinamento
dt_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# 3. Predição
dt_acc = accuracy_score(y_test, dt_model.predict(X_test))
xgb_acc = accuracy_score(y_test, xgb_model.predict(X_test))

print(f"--- Duelo de Modelos (Versão Limpa) ---")
print(f"Acurácia Decision Tree: {dt_acc:.4f}")
print(f"Acurácia XGBoost:       {xgb_acc:.4f}")

--- Duelo de Modelos (Versão Limpa) ---
Acurácia Decision Tree: 0.9474
Acurácia XGBoost:       0.9561


## Desafio 5: "Early Stopping" e Salvamento (Engenharia de ML)

In [ ]:
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Dataset
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Instanciar e Treinar com Early Stopping
model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=42, early_stopping_rounds=10)

print("Iniciando treinamento com Early Stopping...")
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

print(f"Treino parou na iteração: {model.best_iteration}")
print(f"Melhor score (RMSE): {model.best_score:.4f}")

# 2. Salvar o modelo
model_filename = "meu_modelo.json"
model.save_model(model_filename)
print(f"Modelo salvo em '{model_filename}'")

# 3. Carregar o modelo e testar
loaded_model = xgb.XGBRegressor()
loaded_model.load_model(model_filename)

# Prova de funcionamento
sample_pred = loaded_model.predict(X_test[:5])
print(f"\nPredições do modelo carregado (primeiros 5 exemplos): {sample_pred}")

Iniciando treinamento com Early Stopping...
Treino parou na iteração: 541
Melhor score (RMSE): 0.4550
Modelo salvo em 'meu_modelo.json'

Predições do modelo carregado (primeiros 5 exemplos): [0.563204  0.9372465 5.197126  2.5960627 2.4092705]
